In [ ]:
#n this v2, don't quadruple the final layer
#in this v3, lower the weight decay
#in this v5, attempt class balance
#v5 worked! In this v7, try increasing the batch size
#v8: batch size increase may have been slightly detrimental.
    #- go back to batch size 33
    #- here experiment with no deltas
#v9 - deltas add no value to performance. Here try a substantially smaller model
    #-result: hardly lost an accuracy!
#v10 put batch size back done and wd back up

#this v2 - four time larger model

#v3: change kernel for penultimate layer to 3x3 not 1x1

#v4: v3 seemed to help slightly
# - try lwer weight decay, and load spectrograms from disk

#v5: use larger spectrograms

In [1]:
# select a GPU
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "5"


In [2]:
#imports 
import sys
import numpy as np
import matplotlib.pyplot as plt
import h5py
import pandas as pd
import librosa
import soundfile as sound

import tensorflow
from tensorflow.keras.optimizers import SGD

print("Librosa version = ",librosa.__version__)
print("Pysoundfile version = ",sound.__version__)
print("keras version = ",tensorflow.keras.__version__)
print("tensorflow version = ",tensorflow.__version__)

sys.path.insert(0, "../")
from NNets import model_resnet_updated

from DCASE_training_functions import LR_WarmRestart, MixupGenerator



/home/mdmcdonn/anaconda3/envs/April2019/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/mdmcdonn/anaconda3/envs/April2019/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/mdmcdonn/anaconda3/envs/April2019/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/mdmcdonn/anaconda3/envs/A

Librosa version =  0.6.3
Pysoundfile version =  0.10.2
keras version =  2.2.4-tf
tensorflow version =  1.13.1


In [3]:
BasePath = '../../Data/TAU-urban-acoustic-scenes-2020-3class-development/'
TrainFile = BasePath + 'evaluation_setup/fold1_train.csv'
ValFile = BasePath + 'evaluation_setup/fold1_evaluate.csv'
sr = 48000
num_audio_channels = 2

In [4]:
  
SampleDuration = 10

#log-mel spectrogram parameters
NumFreqBins = 256
NumFFTPoints = 4996
HopLength = int(NumFFTPoints/4)
NumTimeBins = int(np.ceil(SampleDuration*sr/HopLength))

#training parameters
init_lr = 0.1
batch_size = 30 #divisible by 3, due to class balance strategy
num_epochs = 126
mixup_alpha = 0.4
crop_length = 400


In [5]:


#load filenames and labels
dev_train_df = pd.read_csv(TrainFile,sep='\t', encoding='ASCII')
dev_val_df = pd.read_csv(ValFile,sep='\t', encoding='ASCII')
wavpaths_train = dev_train_df['filename'].tolist()
wavpaths_val = dev_val_df['filename'].tolist()
y_train_labels =  dev_train_df['scene_label'].astype('category').cat.codes.values
y_val_labels =  dev_val_df['scene_label'].astype('category').cat.codes.values

ClassNames = np.unique(dev_train_df['scene_label'])
NumClasses = len(ClassNames)

y_train = tensorflow.keras.utils.to_categorical(y_train_labels, NumClasses)
y_val = tensorflow.keras.utils.to_categorical(y_val_labels, NumClasses)



In [6]:
ClassNames

array(['indoor', 'outdoor', 'transportation'], dtype=object)

In [7]:
LM_train=np.load('Task1b_LM_train_256_4096.npy')
LM_val=np.load('Task1b_LM_val_256_4096.npy')

In [8]:
LM_train = np.log(LM_train+1e-8)
LM_val = np.log(LM_val+1e-8)

In [9]:
LM_train.shape,LM_val.shape

((9185, 256, 469, 2), (4185, 256, 469, 2))

In [10]:


#create and compile the model
wd = 5e-4
num_filters=25

model = model_resnet_updated(NumClasses,
                     input_shape =[NumFreqBins,None,num_audio_channels], 
                     num_filters =num_filters,
                     wd=wd,binarise_weights=True)
model.compile(loss='categorical_crossentropy',
              optimizer =SGD(lr=init_lr,decay=0, momentum=0.9, nesterov=False),
              metrics=['accuracy'])

model.summary()



Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, None, 2) 0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 128, None, 2) 0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 128, None, 2) 0           input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1 (BatchNo (None, 128, None, 2) 8           lambda[0][0]                     
_____________________________________

In [11]:

#create data generator
TrainDataGen = MixupGenerator(LM_train, 
                              y_train, 
                              batch_size=batch_size,
                              alpha=mixup_alpha,
                              crop_length=crop_length,
                              UseBalance=False)

steps_per_epoch =TrainDataGen.__len__()
lr_scheduler = LR_WarmRestart(nbatch=steps_per_epoch,
                              initial_lr=init_lr, min_lr=init_lr*1e-4,
                              epochs_restart = [1.0,3.0, 7.0, 15.0, 31.0, 63.0,127.0,255.0]) 
callbacks = [lr_scheduler]





In [ ]:
#train the model

history = model.fit_generator(TrainDataGen,
                              validation_data=(LM_val, y_val),
                              epochs=num_epochs, 
                              verbose=1, 
                              workers=1,
                              max_queue_size = 100,
                              callbacks=callbacks,
                              steps_per_epoch=steps_per_epoch
                              )

Instructions for updating:
Use tf.cast instead.

 Start of Epoch Learning Rate = 0.100000
Epoch 1/126
4185/4185 [==============================] - 29s 7ms/sample - loss: 2.7189 - acc: 0.7811

 End of Epoch Learning Rate = 0.050005
306/306 [==============================] - 220s 719ms/step - loss: 3.4817 - acc: 0.6853 - val_loss: 2.7172 - val_acc: 0.7811

 Start of Epoch Learning Rate = 0.050005
Epoch 2/126
4185/4185 [==============================] - 28s 7ms/sample - loss: 2.3387 - acc: 0.8397

 End of Epoch Learning Rate = 0.000010
306/306 [==============================] - 216s 704ms/step - loss: 2.5603 - acc: 0.8065 - val_loss: 2.3391 - val_acc: 0.8397

 Start of Epoch Learning Rate = 0.100000
Epoch 3/126
4185/4185 [==============================] - 27s 7ms/sample - loss: 1.6456 - acc: 0.8010

 End of Epoch Learning Rate = 0.085357
306/306 [==============================] - 215s 704ms/step - loss: 2.1222 - acc: 0.7847 - val_loss: 1.6439 - val_acc: 0.8010

 Start of Epoch Learning Ra

In [ ]:
model.save_weights('Models/Task1B_Final_weights_v5_noundersampling.h5') #damn - I overwrote these!

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(history.history['val_acc'])
plt.plot(history.history['acc'])
plt.ylim([0.8,0.98])

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(np.arange(0,len(lr_scheduler.lr_used))/np.ceil(LM_train.shape[0]/batch_size),lr_scheduler.lr_used,'-')
plt.xlabel('epoch number')
plt.ylabel('learning rate')
plt.show()

In [ ]:
history.history['val_acc'][13],history.history['val_acc'][29],history.history['val_acc'][61],history.history['val_acc'][125],history.history['val_acc'][253],history.history['val_acc'][509]

In [ ]:
max(history.history['val_acc'])

In [ ]:
plt.plot(history.history['val_acc'][450::])
plt.plot(history.history['acc'][450::])

plt.plot(history.history['val_acc'][0:15])
plt.plot(history.history['acc'][0:15])

In [ ]:
import DCASE_plots
#importlib.reload(DCASE_plots)
from DCASE_plots import plot_confusion_matrix

In [ ]:
y_pred = model.predict(LM_val)

In [ ]:
y_pred.shape

In [ ]:
#too many predictions for indoor and outdoor
PC_correct =sum( np.argmax(y_val,axis=-1)==np.argmax([1,0.7,0.5]*y_pred,axis=-1))/y_pred.shape[0]
PC_correct

In [ ]:
plot_confusion_matrix(np.argmax(y_val,axis=-1), np.argmax(y_pred,axis=-1), dev_train_df['scene_label'].unique().tolist(),
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues)

In [ ]:
plot_confusion_matrix(np.argmax(y_val,axis=-1), np.argmax([1,0.7,0.5]*y_pred,axis=-1), dev_train_df['scene_label'].unique().tolist(),
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues)

In [ ]:
#get metrics
from sklearn.metrics import confusion_matrix

weightings = [1,1,0.5] #justification for transport to bbe higher is each example was used less in training

y_pred_val=np.argmax(weightings*y_pred,axis=-1)
y_val_labels=np.argmax(y_val,axis=-1)
Overall_accuracy = np.sum(y_pred_val==y_val_labels)/LM_val.shape[0]
print("overall accuracy: ", Overall_accuracy)


conf_matrix = confusion_matrix(y_val_labels,y_pred_val)
conf_mat_norm_recall = conf_matrix.astype('float32')/conf_matrix.sum(axis=1)[:,np.newaxis]
conf_mat_norm_precision = conf_matrix.astype('float32')/conf_matrix.sum(axis=0)[:,np.newaxis]
recall_by_class = np.diagonal(conf_mat_norm_recall)
precision_by_class = np.diagonal(conf_mat_norm_precision)
mean_recall = np.mean(recall_by_class)
mean_precision = np.mean(precision_by_class)

print("per-class accuracy (recall): ",recall_by_class)
print("per-class precision: ",precision_by_class)
print("mean per-class recall: ",mean_recall)
print("mean per-class precision: ",mean_precision)


In [ ]:
#see if batch norm updates well with  this:
wd = 5e-4
num_filters=25

model = model_resnet_updated(NumClasses,
                     input_shape =[NumFreqBins,469,num_audio_channels], 
                     num_filters =num_filters,
                     wd=wd,binarise_weights=True)
model.compile(loss='categorical_crossentropy',
              optimizer =SGD(lr=1e-3,decay=0, momentum=0.9, nesterov=False),
              metrics=['accuracy'])

model.summary()

model.load_weights('Models/Task1B_Final_weights_v5.h5')

In [ ]:
hh = model.fit(LM_train, y_train, batch_size=18,
#validation_data=(LM_val, y_val),
              
                              epochs=5, 
                              verbose=1, 
                              workers=1,
                              max_queue_size = 100,
                              
                              steps_per_epoch=int(np.ceil(LM_train.shape[0]/batch_size))
                              )